### 0. 라이브러리 불러오기

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import regex
from scipy.stats import hmean

### 1. Book-Crossing 데이터셋 불러오기


#### 활용 데이터셋 : Book-Crossing Dataset

- 출처 :
    <br>Ziegler, C. N., McNee, S. M., Konstan, J. A., & Lausen, G. (2005, May). **Improving recommendation lists through topic diversification.** In Proceedings of the 14th international conference on World Wide Web (pp. 22-32). https://doi.org/10.1145/1060745.1060754
    

* 데이터셋 개요 :
<br>&ensp;&ensp;Book-Crossing (BX) 데이터셋은 사용자들이 책에 대한 평가를 남긴 데이터를 포함하는 데이터셋으로, 2004년 8월부터 9월 사이의 4주 동안 Book-Crossing 커뮤니티를 통해 수집되었습니다.
<br>&ensp;&ensp;BX 데이터셋 내에는 익명의 사용자들이 여러 책에 대해 부여한 평점과 함께, 책의 제목, 저자, 출판사 정보, 사용자의 연령, 성별, 직업 등의 메타데이터가 포함되어 있습니다. 본 Book Rating Prediction 대회에서는 이 중 일부를 가공하여 제공하고 있습니다.

  - 데이터셋 구성
    - **학습 데이터**
        - `books.csv` : 149,570개의 책(item)에 대한 정보를 담고 있는 메타데이터
            - `isbn` : isbn이라고 명시되어 있으나, 엄밀하게는 아마존 제품 코드에 해당하므로 칼럼명을 수정하여 제공
            - `language`, `category`, `summary` : 아마존에서 언어/카테고리/요약 정보를 스크래핑하여 추가 제공
            - `images/` : 메타데이터에 표기된 책 표지 이미지의 url을 스크래핑한 이미지 폴더
        - `users.csv` : 68,092명의 사용자(user)에 대한 정보를 담고 있는 메타데이터
        - `train_ratings.csv` : 59,803명의 사용자가 129,777개의 책에 대해 남긴 306,795건의 평점(rating) 데이터
            - 마찬가지로 `isbn`을 `ASIN`으로 수정하여 제공
    - **테스트 데이터 (본 과제에서는 미사용)**
        - `test_ratings.csv` : 26,167명의 사용자가 52,000개의 책에 대해 남긴 76,699건의 평점 데이터
            - 마찬가지로 `isbn`을 `ASIN`으로 수정하여 제공

* 데이터셋 저작권: [CC0: Public Domain](https://creativecommons.org/publicdomain/zero/1.0/deed.ko)

#### 1.1. 데이터셋 다운로드

In [ ]:
# 기본 디렉토리 설정
base_path: str = '~/book/code/'
data_path: str = os.path.join(base_path, 'data/')

# train, test 데이터 불러오기
book_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "books.csv"))
user_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "users.csv"))
train_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train_ratings.csv"))
test_data: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test_ratings.csv"))
sample_submission: pd.DataFrame = pd.read_csv(os.path.join(data_path, "sample_submission.csv"))

#### 1.2. 데이터셋 확인

In [ ]:
print("books shape:", book_data.shape)
print("users shape:", user_data.shape)
print("ratings shape:", train_data.shape)

##### 1.2.1 책 데이터

In [ ]:
# 책 데이터
fname = "books.csv"

print(f"{fname} 데이터셋 정보")
display(pd.concat([book_data.dtypes, book_data.isnull().sum()], axis=1, keys=["dtype", "isnull"]))
print(f"\n{fname} 데이터셋 일부")
display(book_data.head())

In [ ]:
# 유저 데이터 기술통계량 확인
book_data.describe(include="all")

- 책 제목(`book_title`)은 총 135,436개의 제목이 존재한다. 이 중 가장 많이 중복된 책 제목은 18번 등장한다.
    - 이는 출판사(`publisher`)가 달라서,
    - 저자의 미들네임 표기법이 달라서(`book_author`),
    - 출판 연도(`year_of_publication`)가 달라서
    
    등 다양한 이유로 나뉜다.
    
- 저자(`book_author`)는 총 62,058명이 존재한다. 이 중 가장 많은 책을 출판한 사람은 스티븐 킹이다.
    - 하지만 위 예시처럼 이름 표기법의 차이로 같은 인물이지만 다른 문자열로 표시된 경우도 존재하므로 실제 저자의 수는 더 적을 것이다.
    - 한편 저자가 쓴 책의 종류를 확인하고 싶으면 `book_title`의 unique의 수를 세는 것이 바람직하다. 같은 책이라도 출판사, 발행 연도, 언어에 따라 다른 isbn 코드를 갖기 때문이다.
    - 만약 같은 책인 경우 하나의 데이터만 사용하고 싶다면 발행 연도(`year_of_publication`)가 가장 빠른 데이터를 기준으로 삼는 것이 좋을 것 같다. 언어가 다른 경우는 다른 데이터로 취급하는 것이 좋을 것 같다.
- 발행 연도(`year_of_publication`)는 1923년부터 2006년까지는 모두 존재한다. 이전 시점에는 다음 연도들이 존재한다.
    
    [1376, 1378, 1806, 1900, 1901, 1902, 1904, 1906, 1908, 1911, 1920]
    
    
- 출판사(`publisher`)는 11,571개의 출판사가 존재한다.

- 출판 언어(`language`)는 총 26개의 언어가 존재한다.
    
    결측치를 제외한 대다수의 책이 영어로 쓰여있고(95%, 78,823개), 전체 데이터 중 결측치는 67,227개로 두 번째로 많은 비율을 차지한다.
    
    
- 카테고리(`category`)는 총 4,292개의 키워드로 구성되어 있다. 가장 많이 나타난 키워드는 Fiction으로 32,956번 나타난다.    

- 요약(`summary`)에는 총 79,872개의 내용이 존재한다. 중복되는 요약이 있음을 확인할 수 있고 가장 많이 중복된 내용은 42번이다.

In [ ]:
# 출판 연도 종류 확인
book_data["year_of_publication"].sort_values()

In [ ]:
# 극단값을 제외한 출판 연도 히스토그램 시각화
sns.histplot(book_data["year_of_publication"].sort_values()[3:], kde=True)  # 1376, 1378, 1806년 제외
plt.show()

In [ ]:
# 언어 종류 확인
book_data["language"].sort_values().unique()

In [ ]:
# 언어 빈도수 확인
book_data["language"].value_counts()

In [ ]:
# 언어 히스토그램 시각화
fig, ax = plt.subplots(figsize=(12, 5))
sns.histplot(book_data["language"])
plt.show()

#### 1.2.1.1 책 데이터 전처리

In [ ]:
def str2list(x: str) -> list:
    '''문자열을 리스트로 변환하는 함수'''
    return x[2:-2].split(", ")


book_data["category"] = book_data["category"].apply(lambda x: str2list(x)[0] if not pd.isna(x) else np.nan)  # 카테고리 대괄호, 따옴표 제거
book_data["language"] = book_data["language"].fillna(book_data["language"].mode()[0])  # 언어 결측치 최빈값으로 대체

In [ ]:
# 카테고리 빈도수 확인
book_data["category"].value_counts()

In [ ]:
# 상위 카테고리 50개 히스토그램 시각화
fig, ax = plt.subplots(figsize=(30, 5))
sns.barplot(book_data["category"].value_counts()[:50])
plt.xticks(rotation=90)
plt.show()

##### 1.2.2 유저 데이터

In [ ]:
# 유저 데이터
fname = "users.csv"

print(f"{fname} 데이터셋 정보")
display(pd.concat([user_data.dtypes, user_data.isnull().sum()], axis=1, keys=["dtype", "isnull"]))
print(f"\n{fname} 데이터셋 일부")
display(user_data.head())

In [ ]:
# 유제 데이터 기술통계량 확인
user_data.describe(include="all")

In [ ]:
# 나이 상자그림 시각화
sns.boxplot(data=user_data, x="age")
plt.show()

In [ ]:
# 나이 히스토그램 시각화
sns.histplot(user_data["age"], kde=True)
plt.show()

#### 1.2.2.1 유저 데이터 전처리

In [ ]:
def split_location(x: str) -> list:
    res = x.split(",")
    res = [i.strip().lower() for i in res]
    res = [regex.sub(r"[^a-zA-Z/ ]", "", i) for i in res]  # remove special characters
    res = [i if i not in ["n/a", ""] else np.nan for i in res]  # change "n/a" into <NA>
    res.reverse()

    # remove duplicates inside list
    for i in range(len(res)-1, 0, -1):
        if res[i] in res[:i]:
            res.pop(i)

    return res


# 나이 결측치를 최빈값으로 대체
user_data["age"] = user_data["age"].fillna(user_data["age"].mode()[0])
# location 정보를 리스트로 변환 및 중복 제거
user_data["location_list"] = user_data["location"].apply(lambda x: split_location(x))
user_data["location_country"] = user_data["location_list"].apply(lambda x: x[0])
user_data["location_state"] = user_data["location_list"].apply(lambda x: x[1] if len(x) >= 2 else np.nan)
user_data["location_city"] = user_data["location_list"].apply(lambda x: x[2] if len(x) >= 3 else np.nan)

# 만일 지역 정보의 뒷부분(주 또는 도시)은 주어졌으나 지역 정보 앞부분(국가 또는 주)이 없는 경우, 최빈값으로 대체
for idx, row in user_data.iterrows():
    if (not pd.isna(row["location_state"])) and pd.isna(row["location_country"]):
        fill_country = user_data[user_data["location_state"] == row["location_state"]]["location_country"].mode()
        fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
        user_data.loc[idx, "location_country"] = fill_country
    elif (not pd.isna(row["location_city"])) and pd.isna(row["location_state"]):
        if not pd.isna(row["location_country"]):
            fill_state = user_data[(user_data["location_country"] == row["location_country"])
                                & (user_data["location_city"] == row["location_city"])]["location_state"].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            user_data.loc[idx, "location_state"] = fill_state
        else:
            fill_state = user_data[user_data["location_city"] == row["location_city"]]["location_state"].mode()
            fill_state = fill_state[0] if len(fill_state) > 0 else np.nan
            fill_country = user_data[user_data["location_city"] == row["location_city"]]["location_country"].mode()
            fill_country = fill_country[0] if len(fill_country) > 0 else np.nan
            user_data.loc[idx, "location_country"] = fill_country
            user_data.loc[idx, "location_state"] = fill_state

# 최종 결측치 처리 -> "unknown"으로 대체
user_data = user_data.fillna("unknown")

In [ ]:
user_data["location_country"].value_counts()

In [ ]:
# 상위 국가 50개 히스토그램 시각화
fig, ax = plt.subplots(figsize=(30, 5))
sns.barplot(user_data["location_country"].value_counts()[:50])
plt.xticks(rotation=90)
plt.show()

전처리 후 시각화한 결과 `location`에 다음과 같은 문제들이 발생한다.
1. `location`에 이상하게 기입된 내용이 존재한다.
2. `country`에 `state` 정보가 들어간 경우도 있다. 최빈값으로 대체된 경우이거나 처음부터 country 자리에 `state`가 위치한 경우로 예상된다.




In [ ]:
# 제대로 기입되지 않은 location 정보 존재
user_data[user_data["location"].apply(lambda x:x.find("universe" or "everywhere and anywhere")) != -1]

##### 1.2.3 평점 데이터

In [ ]:
# 평점 데이터
fname = "train_ratings.csv"

print(f"{fname} 데이터셋 정보")
display(pd.concat([train_data.dtypes, train_data.isnull().sum()], axis=1, keys=["dtype", "isnull"]))
print(f"\n{fname} 데이터셋 일부")
display(train_data.head())

In [ ]:
# 중복되지 않은 "user_id"의 수로 전체 유저 수 구하기
n_user = len(train_data["user_id"].unique())
# 중복되지 않은 "isbn"의 수로 전체 책 수 구하기
n_item = len(train_data["isbn"].unique())

print("사용자 수:", n_user, ", 책 수:", n_item)

In [ ]:
# 평점 데이터 기술통계량 확인
train_data[["rating"]].describe()

In [ ]:
# 평점 히스토그램 시각화
sns.countplot(data=train_data, x="rating")
plt.show()

##### 1.3 데이터 병합

In [ ]:
# book, user, rating 데이터 병합
df = pd.merge(train_data, book_data, on="isbn", how="left")
df = pd.merge(df, user_data, on="user_id", how="left")
df

#### Idea. 유저별 평균 평점 피처

평균을 일정 이상 매긴 유저는 평점 분포를 확인할 수 있고, 평균 평점도 계산할 수 있다. 

새로운 책을 읽으면 어떤 패턴으로 평점을 매길지 유추하기 위해 평균 평점을 활용하고 싶고, 이를 새로운 피처로 추가해보고자 한다.

한편 평점(`rating`)을 활용하는 순간 test_data에서 처음으로 등장하는 유저의 경우 평점 정보가 전혀 없으므로 이를 해결해줘야 한다.

In [ ]:
# 평점을 100개 이상 매긴 유저의 id만 필터링
heavy_user_idx = df.groupby("user_id")["rating"].count()
heavy_user_idx = heavy_user_idx[heavy_user_idx >= 100].index

# 해당 유저의 데이터만 필터링
heavy_user_df = df[df["user_id"].isin(heavy_user_idx)]
heavy_user_df

In [ ]:
# 유저별 평점 분포 확인 (유저별 rating 값의 빈도수 계산) -> 매긴 적 없는 평점 level에는 0으로 대체
rating_distribution = heavy_user_df.groupby("user_id")["rating"].value_counts().unstack(fill_value=0)
rating_distribution

In [ ]:
# 유저별 평점의 산술평균과 조화평균 계산
heavy_user_averages = heavy_user_df.groupby("user_id")["rating"].agg(
    arithmetic_mean=np.mean,
    harmonic_mean=lambda x: hmean(x) if (x > 0).all() else np.nan  # 조화평균은 0이 아닌 값만 포함해야 한다. (역수를 취해야 하므로)
)
pd.concat((rating_distribution, heavy_user_averages), axis=1).head(20)

#### Cold Start 문제

In [ ]:
# train과 test의 user_id를 집합(set) 형태로 저장
train_users = set(train_data["user_id"].unique())
test_users = set(test_data["user_id"].unique())

# test에만 존재하는 user_id 추출
new_users_in_test = test_users - train_users
print(f"Number of new users in test set: {len(new_users_in_test)}")
# print("New users in test set:", new_users_in_test)
print(f"cold start ratio: {len(new_users_in_test) / len(test_data)}")

In [ ]:
# user_data에서 test에만 존재하는 user_id 추출
new_users_data = user_data[user_data["user_id"].isin(new_users_in_test)]
new_users_data

In [ ]:
# test에만 존재하는 유저의 나이, 위치 정보(국가) 목록 확인
demography = new_users_data[["age", "location_country"]].drop_duplicates()
demography

In [ ]:
# df에서 demography에 있는 (age, location_country) 조합만 필터링
filtered_df = df.merge(demography, on=["age", "location_country"], how="inner")
filtered_df

In [ ]:
# 유저 수 확인
print(user_data["user_id"].nunique())
print(df["user_id"].nunique())
print(filtered_df["user_id"].nunique())

In [ ]:
# filtered_df에서 demography별 평균 평점 계산
filtered_df.groupby(["age", "location_country"])["rating"].agg(
    arithmetic_mean=np.mean,
    harmonic_mean=lambda x: hmean(x) if (x > 0).all() else np.nan  # 조화평균은 0이 아닌 값만 포함해야 한다. (역수를 취해야 하므로)
)